In [1]:
import mne
import openneuro
from pathlib import Path

In [2]:
# Указываем параметры для вашего датасета
dataset = "ds005385"
subject = "001"  # или любой другой subject ID из этого датасета

# Создаем путь для скачивания
bids_root = Path("./data") / dataset  # скачаем в папку data/ds005385 текущей директории
bids_root.mkdir(parents=True, exist_ok=True)

print(f"Скачиваем датасет {dataset}, субъект {subject}...")
print(f"Файлы будут сохранены в: {bids_root.absolute()}")

# Скачиваем только данные указанного субъекта
openneuro.download(
    dataset=dataset,
    target_dir=bids_root,
    include=[f"sub-{subject}"]  # скачиваем только одного субъекта
)

print("Скачивание завершено!")

Скачиваем датасет ds005385, субъект 001...
Файлы будут сохранены в: C:\Users\me\Documents\Projects_Python314\data\ds005385

👋 Hello! This is openneuro-py 2025.1.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds005385 …


📁 Traversing directories for ds005385 : 0 entities [00:00, ? entities/s]

log_error - https://openneuro.org/crn/graphql: 502 Server Error: Bad Gateway for url: https://openneuro.org/crn/graphql
C:\Users\me\AppData\Local\Programs\Python\Python314\Lib\site-packages\openneuro\_download.py:220: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(retry_backoff)  # pyright: ignore[reportUnusedCoroutine]


  Request timed out while fetching metadata, retrying …


log_error - https://openneuro.org/crn/graphql: 502 Server Error: Bad Gateway for url: https://openneuro.org/crn/graphql


...

  Request timed out while fetching metadata, retrying …


log_error - https://openneuro.org/crn/graphql: 502 Server Error: Bad Gateway for url: https://openneuro.org/crn/graphql


  Request timed out while fetching metadata, retrying …


log_error - https://openneuro.org/crn/graphql: 502 Server Error: Bad Gateway for url: https://openneuro.org/crn/graphql


RuntimeError: Timeout when trying to fetch metadata.

In [ ]:
bids_root = Path("./data/ds005385")

# Проверяем структуру скачанных файлов
print("Структура скачанного датасета:")
for path in bids_root.rglob("*"):
    if path.is_file():
        print(f"  {path.relative_to(bids_root)}")
        # Проверяем размер файла
        print(f"    Размер: {path.stat().st_size} байт")

# Проверяем конкретно проблемный файл
problem_file = bids_root / "sub-001/ses-1/eeg/sub-001_ses-1_task-EyesOpen_acq-post_eeg.edf"
if problem_file.exists():
    print(f"✓ Проблемный файл скачан: {problem_file.stat().st_size} байт")
else:
    print(f"✗ Проблемный файл отсутствует: {problem_file}")

In [ ]:
from mne_bids import BIDSPath, read_raw_bids

bids_root = "./data/ds005385"

# Создаем BIDSPath с указанием acquisition
bids_path = BIDSPath(
    subject='001',
    session='1',
    task='EyesClosed',
    acquisition='post',  # добавляем параметр acquisition
    datatype='eeg',
    root=bids_root
)

print("Пробуем загрузить данные EyesClosed acq-post...")

try:
    raw = read_raw_bids(bids_path)
    print("✓ Данные успешно загружены!")
    print(raw)
    
    # Проверяем и устанавливаем расположение электродов
    if raw.info['dig'] is None:
        print("Устанавливаем стандартное расположение электродов...")
        raw.set_montage('standard_1020')
    
    # Показываем сырые данные
    print("Открываем просмотр сырых данных...")
    raw.plot()
    
except Exception as e:
    print(f"Ошибка при загрузке: {e}")

In [ ]:
# Проверим основные параметры данных
print("Информация о данных:")
print(f"Количество каналов: {len(raw.ch_names)}")
print(f"Частота дискретизации: {raw.info['sfreq']} Гц")
print(f"Длительность записи: {raw.times[-1]:.2f} секунд")
print(f"Типы каналов: {set(ch['kind'] for ch in raw.info['chs'])}")

# Посмотрим на имена каналов
print("\nПервые 10 каналов:")
for ch_name in raw.ch_names[:10]:
    print(f"  {ch_name}")

# Проверим, есть ли у нас координаты для каналов после установки montage
if raw.info['dig'] is not None:
    print("\n✓ Расположение электродов установлено")
else:
    print("\n✗ Расположение электродов не установлено")

In [ ]:
# Проверим, какие каналы у нас есть и отберем только EEG
print("Анализ каналов:")
eeg_picks = mne.pick_types(raw.info, eeg=True, eog=False, ecg=False, stim=False)
print(f"EEG каналов: {len(eeg_picks)}")
print(f"Стимульные каналы: {len(raw.info['chs']) - len(eeg_picks)}")

# Посмотрим на имена EEG каналов
eeg_ch_names = [raw.ch_names[i] for i in eeg_picks]
print("EEG каналы:")
for i in range(0, len(eeg_ch_names), 8):
    print("  " + ", ".join(eeg_ch_names[i:i+8]))

In [ ]:
# Загружаем данные для анализа
data = raw.get_data(picks=eeg_picks)
print(f"Данные для анализа микросостояний: {data.shape}")

# Извлекаем пики GFP
from pycrostates.preprocessing import extract_gfp_peaks
gfp_peaks = extract_gfp_peaks(data)
print(f"Пики GFP: {gfp_peaks.shape}")

# Кластеризация с разным количеством кластеров для сравнения
from pycrostates.clustering import KMeans
import numpy as np

# Попробуем разное количество кластеров
cluster_range = [3, 4, 5, 6]
models = {}

for n_clusters in cluster_range:
    print(f"\nКластеризация с {n_clusters} кластерами...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(gfp_peaks, n_jobs=1)
    models[n_clusters] = kmeans
    
    # Визуализируем карты микросостояний
    fig = kmeans.plot()
    plt.suptitle(f'Карты микросостояний - {n_clusters} кластеров')
    plt.show()